In [2]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import newton

def black_scholes_price(S, K, T, r, sigma, q, option_type='call'):
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        price = S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r*T) * norm.cdf(-d2) - S * np.exp(-q*T) * norm.cdf(-d1)
    return price

def implied_volatility(S, K, T, r, q, market_price, option_type='call'):
    # 定义目标函数：模型价格与市场价格的差值
    def objective(sigma):
        return black_scholes_price(S, K, T, r, sigma, q, option_type) - market_price
    
    # 初始猜测范围（通常取0.1~1.0）
    initial_guess = 0.3
    return newton(objective, x0=initial_guess, maxiter=100)

# 示例：计算隐含波动率
S = 100.0       # 标的资产价格
K = 100.0       # 行权价
T = 1.0         # 到期时间（年）
r = 0.05        # 无风险利率
q = 0.01        # 股息率
market_price = 10.0  # 假设市场报价为10美元
iv = implied_volatility(S, K, T, r, q, market_price, 'call')
print(f"隐含波动率: {iv:.2%}")

隐含波动率: 20.46%


In [9]:
K1 = max([k for k in K_grid if k <= K])
T1 = max([t for t in T_grid if t <= T])
K_grid.index(K1)
IV_grid
#IV_grid[K_grid.index(K1), T_grid.index(T1)]

[[0.3, 0.28, 0.25, 0.22],
 [0.28, 0.25, 0.23, 0.2],
 [0.25, 0.23, 0.2, 0.18],
 [0.23, 0.22, 0.19, 0.17],
 [0.22, 0.2, 0.18, 0.16]]

In [32]:
def bilinear_interpolation(K, T, K_grid, T_grid, IV_grid):
    # 找到最近的四个网格点
    K1 = max([k for k in K_grid if k <= K])
    K2 = min([k for k in K_grid if k >= K])
    T1 = max([t for t in T_grid if t <= T])
    T2 = min([t for t in T_grid if t >= T])
    
    # 获取四个点的波动率
    IV11 = IV_grid[K_grid.index(K1)][T_grid.index(T1)]
    IV12 = IV_grid[K_grid.index(K1)][T_grid.index(T2)]
    IV21 = IV_grid[K_grid.index(K2)][T_grid.index(T1)]
    IV22 = IV_grid[K_grid.index(K2)][T_grid.index(T2)]
    
    # 计算插值权重
    weight_K = (K - K1) / (K2 - K1) if K2 != K1 else 0.0
    weight_T = (T - T1) / (T2 - T1) if T2 != T1 else 0.0
    
    # 双线性插值公式
    IV = (IV11 * (1 - weight_K) * (1 - weight_T) +
          IV21 * weight_K * (1 - weight_T) +
          IV12 * (1 - weight_K) * weight_T +
          IV22 * weight_K * weight_T)
    return IV

# 示例：插值计算 K=81, T=0.75 的波动率
K_grid = [80, 90, 100, 110, 120]
T_grid = [0.25, 0.5, 1.0, 2.0]
IV_grid = [
    [0.30, 0.28, 0.25, 0.22],   # K=80
    [0.28, 0.25, 0.23, 0.20],   # K=90
    [0.25, 0.23, 0.20, 0.18],   # K=100
    [0.23, 0.22, 0.19, 0.17],   # K=110
    [0.22, 0.20, 0.18, 0.16],   # K=120
]


for k in range(80, 91):
    IV_interp = bilinear_interpolation(k, 0.25, K_grid, T_grid, IV_grid)
    print(f"插值波动率（K={k}, T=0.25）: {IV_interp:.2%}")


插值波动率（K=80, T=0.25）: 30.00%
插值波动率（K=81, T=0.25）: 29.80%
插值波动率（K=82, T=0.25）: 29.60%
插值波动率（K=83, T=0.25）: 29.40%
插值波动率（K=84, T=0.25）: 29.20%
插值波动率（K=85, T=0.25）: 29.00%
插值波动率（K=86, T=0.25）: 28.80%
插值波动率（K=87, T=0.25）: 28.60%
插值波动率（K=88, T=0.25）: 28.40%
插值波动率（K=89, T=0.25）: 28.20%
插值波动率（K=90, T=0.25）: 28.00%
